ML соревнование по предсказанию уровня сеймоустройчивости здания (классификация) на [DrivenData](https://www.drivendata.org/competitions/57/nepal-earthquake/). Данные по землетрясению в Непале. <br><br>
Текущее место: **46/2804** <br>
Текущий скор: **0.7501** (у лидера — 0.7558). <br><br>
**Кратко о данных**: 39 признаков, связанных с кодированной геолокацией, материалами здания, типом организаций в здании, его габаритами, этажностью. <br> **Таргет**: 3 класса (1 - худшая сейсмоустойчивость, 3 - лучшая). Отношение классов ~ 1 : 5.5 : 3.5. <br><br>
**Train**: ~260600 зданий.  <br>
**Test**: ~86000 зданий. <br> 
**Метрика**: F1-micro.

In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score, confusion_matrix

from lightgbm import LGBMClassifier

Изначальный вид данных:

In [ ]:
df = pd.read_csv('train_values.csv')
df.head()

,building_id,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,legal_ownership_status,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,has_secondary_use_rental,has_secondary_use_institution,has_secondary_use_school,has_secondary_use_industry,has_secondary_use_health_post,has_secondary_use_gov_office,has_secondary_use_use_police,has_secondary_use_other
0,802906,6,487,12198,2,30,6,5,t,r,n,f,q,t,d,1,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
1,28830,8,900,2812,2,10,8,7,o,r,n,x,q,s,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
2,94947,21,363,8973,2,10,5,5,t,r,n,f,x,t,d,0,1,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
3,590882,22,418,10694,2,10,6,5,t,r,n,f,x,s,d,0,1,0,0,0,0,1,1,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0
4,201944,11,131,1488,3,30,8,9,t,r,n,f,x,s,d,1,0,0,0,0,0,0,0,0,0,0,v,1,0,0,0,0,0,0,0,0,0,0,0


Предподготовка:

- Посмотрели распределение таргета и некоторых численных столбцов.
- Детальнее проанализировали данные. Для всех выводов хватило базовых популярных функций: *describe*, *nunique*, *unique*, *isna*.

In [ ]:
def target_encoding(df):
  for column in obj_columns:
    lens = df.groupby(column)['target'].agg('count') # смотрим, как распределён таргет при каждом значении выбранного столбца
    lens_1 = df[df.target == 1].groupby(column)['target'].agg('count')
    lens_2 = df[df.target == 2].groupby(column)['target'].agg('count')
    lens_3 = df[df.target == 3].groupby(column)['target'].agg('count')
    ratio_1 = lens_1 / lens
    ratio_2 = lens_2 / lens
    ratio_3 = lens_3 / lens
    df[column + '_1'] = df.merge(ratio_1, how = 'left', on = column)['target_y']
    df[column + '_2'] = df.merge(ratio_2, how = 'left', on = column)['target_y']
    df[column + '_3'] = df.merge(ratio_3, how = 'left', on = column)['target_y']
    df = df.fillna(0.333)
  print('Train encoding: done')
  return df

def target_encoding_test(df_test): # то же самое, что и предыдущая функция, но для теста
  for column in obj_columns:
    lens = df.groupby(column)['target'].agg('count')
    lens_1 = df[df.target == 1].groupby(column)['target'].agg('count')
    lens_2 = df[df.target == 2].groupby(column)['target'].agg('count')
    lens_3 = df[df.target == 3].groupby(column)['target'].agg('count')
    ratio_1 = lens_1 / lens
    ratio_2 = lens_2 / lens
    ratio_3 = lens_3 / lens
    lens_transform = lens / len(df)
    df_test[column + '_1'] = df_test.merge(ratio_1, how = 'left', on = column)['target']
    df_test[column + '_2'] = df_test.merge(ratio_2, how = 'left', on = column)['target']
    df_test[column + '_3'] = df_test.merge(ratio_3, how = 'left', on = column)['target']
    df_test = df_test.fillna(0.333)
  print('Test encoding: done')
  return df_test

def preprocessing(df):
  df.drop(columns = 'building_id', inplace = True)
  df['percentage_production'] = df.area_percentage * df.height_percentage
  df['families_per_floor'] = df.count_families / df.count_floors_pre_eq
  df['geo_level_id_2_round_10'] = df.geo_level_2_id.astype('int').round(-1)
  df['geo_level_id_2_round_100'] = df.geo_level_2_id.astype('int').round(-2)
  df['geo_level_id_3_round_10'] = df.geo_level_3_id.astype('int').round(-1)
  df['geo_level_id_3_round_100'] = df.geo_level_3_id.astype('int').round(-2)
  print('Preprocessing: done')
  return df

def reduce_mem_usage(df, verbose=True): # готовая функция, взята с Kaggle
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [ ]:
df = pd.read_csv('train_values.csv')
df_test = pd.read_csv('test_values.csv')
labels = pd.read_csv('train_labels.csv')
df['target'] = labels.damage_grade.values

# список столбцов для преобразования
obj_columns = df.loc[:, 'land_surface_condition':'plan_configuration'].columns.tolist()
obj_columns.append('geo_level_1_id')
obj_columns.append('geo_level_2_id')
obj_columns.append('count_floors_pre_eq')
obj_columns.append('legal_ownership_status')
obj_columns.append('age')

# обработка
df = preprocessing(df)
df_test = preprocessing(df_test)
df = target_encoding(df)
df_test = target_encoding_test(df_test)

# столбцы на удаление
obj_columns = df.loc[:, 'land_surface_condition':'plan_configuration'].columns.tolist()
obj_columns.append('legal_ownership_status')
obj_columns.append('has_secondary_use_health_post')
obj_columns.append('has_secondary_use_gov_office')
obj_columns.append('has_secondary_use_use_police')
df.drop(columns = obj_columns, inplace = True)
df_test.drop(columns = obj_columns, inplace = True)

X = df.copy()
X_test = df_test.copy()
X.drop(columns = 'target', inplace = True)
y = df.target

X = reduce_mem_usage(X)
X_test = reduce_mem_usage(X_test)

# фичи на удаление, определённые как самые «бесполезные» через feature_importance
to_drop = ['has_secondary_use_school', 'has_secondary_use_institution', 'land_surface_condition_2', 'roof_type_3', 'legal_ownership_status_3', 'has_secondary_use_industry', \
           'position_3', 'other_floor_type_2', 'foundation_type_3', 'has_secondary_use_other', 'geo_level_id_2_round_100', 'plan_configuration_3', 'has_secondary_use_rental',
           'count_floors_pre_eq_3']
X.drop(columns = to_drop, inplace = True)
X_test.drop(columns = to_drop, inplace = True)

Preprocessing: done
Preprocessing: done
Train encoding: done
Test encoding: done
Mem. usage decreased to 28.33 Mb (79.3% reduction)
Mem. usage decreased to  9.44 Mb (79.3% reduction)


Рабочие данные:

In [ ]:
X.head(3)

,geo_level_1_id,geo_level_2_id,geo_level_3_id,count_floors_pre_eq,age,area_percentage,height_percentage,has_superstructure_adobe_mud,has_superstructure_mud_mortar_stone,has_superstructure_stone_flag,has_superstructure_cement_mortar_stone,has_superstructure_mud_mortar_brick,has_superstructure_cement_mortar_brick,has_superstructure_timber,has_superstructure_bamboo,has_superstructure_rc_non_engineered,has_superstructure_rc_engineered,has_superstructure_other,count_families,has_secondary_use,has_secondary_use_agriculture,has_secondary_use_hotel,percentage_production,families_per_floor,geo_level_id_2_round_10,geo_level_id_3_round_10,geo_level_id_3_round_100,land_surface_condition_1,land_surface_condition_3,foundation_type_1,foundation_type_2,roof_type_1,roof_type_2,ground_floor_type_1,ground_floor_type_2,ground_floor_type_3,other_floor_type_1,other_floor_type_3,position_1,position_2,plan_configuration_1,plan_configuration_2,geo_level_1_id_1,geo_level_1_id_2,geo_level_1_id_3,geo_level_2_id_1,geo_level_2_id_2,geo_level_2_id_3,count_floors_pre_eq_1,count_floors_pre_eq_2,legal_ownership_status_1,legal_ownership_status_2,age_1,age_2,age_3
0,6,487,12198,2,30,6,5,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,30,0.5,490,12200,12200,0.101318,0.335449,0.04892,0.572754,0.074097,0.582031,0.059509,0.571777,0.368652,0.044708,0.360352,0.080688,0.529297,0.093201,0.569824,0.086487,0.665527,0.248169,0.003704,0.251953,0.744629,0.080383,0.600098,0.092712,0.570312,0.035492,0.579102,0.385254
1,8,900,2812,2,10,8,7,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,56,0.5,900,2810,2800,0.072388,0.361572,0.04892,0.572754,0.074097,0.582031,0.082458,0.584473,0.333252,0.044708,0.360352,0.098328,0.574219,0.093201,0.569824,0.034271,0.446289,0.519531,0.010048,0.492432,0.497559,0.080383,0.600098,0.092712,0.570312,0.112122,0.575195,0.312744
2,21,363,8973,2,10,5,5,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,25,0.5,360,8970,9000,0.101318,0.335449,0.04892,0.572754,0.074097,0.582031,0.059509,0.571777,0.368652,0.078918,0.376221,0.080688,0.529297,0.093201,0.569824,0.021622,0.393311,0.584961,0.082397,0.316406,0.601074,0.080383,0.600098,0.092712,0.570312,0.112122,0.575195,0.312744


<h3>Итоговая модель</h3>

1. В качестве модели выступает Light GBM Classifier.
2. Параметры подбирались вручную.
3. Оценка параметров производилась на кроссвалидации на 5 фолдах. <br>
• Блок кода был абсолютно аналогичным, за исключением, что оценки складывались в список. <br>
• Удобство данных в том, что оценка кроссвалидации достаточно близка с результатом на тестовых данных. <br>
• При оценке на кроссвалидации смотрел не только на средний скор, но и на дисперсию 5 оценок, чтобы оценка была более устойчивой. 
4. При обучении разбивали трейн на 9 фолдов и каждый раз обучались на 8/9 датасета, используя для валидации оставшую 1/9 данных. 
5. Каждое предсказание заносили в как столбец в датасет.
6. Итоговый результат = усреднение 9 предсказаний с округлением. 

In [ ]:
fold = StratifiedKFold(n_splits = 9, random_state = 42)
res = pd.DataFrame()
res_val = []
i = 1
for train_index, test_index in fold.split(X, y):
  X_train, X_val = X.iloc[train_index], X.iloc[test_index]
  y_train, y_val = y.iloc[train_index], y.iloc[test_index]
  lgbc = LGBMClassifier(objective = 'multiclass',
                        n_estimators = 2500,
                        min_sum_hessian_in_leaf = 1,
                        lambda_l1 = 8,
                        lambda_l2 = 6,
                        min_data_in_leaf = 40,
                        learning_rate = 0.12)
  lgbc.fit(X_train, y_train, eval_set = [(X_val, y_val)], early_stopping_rounds = 50, verbose = 250)
  pred = lgbc.predict(X_test)
  print('Fold {} done. F1 score on validation set: {}'.format(i, f1_score(y_val, lgbc.predict(X_val), average = 'micro')))
  res_val.append(f1_score(y_val, lgbc.predict(X_val), average = 'micro'))
  res[i] = pred
  i += 1
print('Training: done. Mean validation score: {}'.format(np.mean(res_val)))

sub = res.mean(axis = 1).round().astype('int')
sf = pd.read_csv('submission_format.csv', index_col = 'building_id')
my_submission = pd.DataFrame(data = sub.values, columns = sf.columns, index = sf.index)
res_name = 'prediction.csv'
my_submission.to_csv(res_name)
print('Saved file {}'.format(res_name))

Training until validation scores don't improve for 50 rounds.
[250]	valid_0's multi_logloss: 0.580963
[500]	valid_0's multi_logloss: 0.571705
[750]	valid_0's multi_logloss: 0.56715
[1000]	valid_0's multi_logloss: 0.564398
[1250]	valid_0's multi_logloss: 0.562898
[1500]	valid_0's multi_logloss: 0.561813
[1750]	valid_0's multi_logloss: 0.560893
Early stopping, best iteration is:
[1828]	valid_0's multi_logloss: 0.560769
Fold 1 done. F1 score on validation set: 0.75089791407653
Training until validation scores don't improve for 50 rounds.
[250]	valid_0's multi_logloss: 0.576729
[500]	valid_0's multi_logloss: 0.566817
[750]	valid_0's multi_logloss: 0.561916
[1000]	valid_0's multi_logloss: 0.558915
[1250]	valid_0's multi_logloss: 0.557087
[1500]	valid_0's multi_logloss: 0.555982
[1750]	valid_0's multi_logloss: 0.554982
[2000]	valid_0's multi_logloss: 0.554618
Early stopping, best iteration is:
[2067]	valid_0's multi_logloss: 0.554513
Fold 2 done. F1 score on validation set: 0.753902472717226

Важнейшие фичи:

In [ ]:
pd.DataFrame({'feature': X.columns, 'lgb_imp': lgbc.feature_importances_}).sort_values(by = 'lgb_imp', ascending = False).head(25)

,feature,lgb_imp
2,geo_level_3_id,17881
1,geo_level_2_id,14276
46,geo_level_2_id_2,13504
45,geo_level_2_id_1,13122
22,percentage_production,13009
47,geo_level_2_id_3,11096
5,area_percentage,8486
25,geo_level_id_3_round_10,7019
6,height_percentage,5415
0,geo_level_1_id,5371


<h3>Что не сработало<h3>



- XGBoost давал очень слабые результаты.
- Catboost также удалось дотюнить максимум до 0.742.
- Эффективно показывают себя фичи, связанные с target encoding'ом регионов геолокации 1 и 2 (более крупные регионы). Попытка кодировать также 3-й уровень геолокации (более детальный) приводит к резкому улучшению скора на трейне, но только к падению на тесте (очевидно, начинаем переобучаться под трейн). 
- Вместо predict было опробовано predict_proba. Смотрели, где будет лучшая накопленная вероятность после 9 обучений. Предполагалось, что это поможет лучше чувствовать границы между вариантами таргета. Результат был почти эквивалентен выбранной методике (predict + mean + round), но всё же чуть уступал.
- Отсутсвующие значения при target encoding'е заполняются значением 1/3. Пробовал заполнять их более детально (если где-то нет значений - там 0, в остальных - соответствующие доли), прироста это не дало.
- Попытка считать среднее значение таргета по различным данным в столбцах (с предположением, что 1,2 и 3 можно интерпретировать как количественное значение? условную «оценку»).